In [2]:
import os
import pandas as pd
import geopandas as gpd
import plotly.express as px
import ast
%pylab inline

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 40000)
pd.set_option('display.max_rows', None)

df_safegraph_poi = pd.read_csv(os.path.join('..', 'dewey_data_filtered','SafeGraph_POI_Yellowstone_200miRadius.csv.gz'))
df_safegraph_spend = pd.read_csv(os.path.join('..', 'dewey_data_filtered','SafeGraph_SPEND_Yellowstone_200miRadius.csv.gz'))


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
def map_points(fig,df):

    gdf_safegraph_poi = gpd.GeoDataFrame(df,
                                         geometry=gpd.points_from_xy(df.LONGITUDE, df.LATITUDE))
    fig = px.scatter_mapbox(
        gdf_safegraph_poi,
        lat='LATITUDE',
        lon='LONGITUDE',
        hover_name="LOCATION_NAME",
        hover_data=["CITY"],
        zoom=6,
        opacity=0.5)
  
    return fig

In [4]:
def get_spending(df, start_date = '2022-01-01', end_date = '2022-12-31'):
    datetime_start = pd.to_datetime(start_date)
    datetime_end = pd.to_datetime(end_date)

    df_filtered = df.copy()

    df['SPEND_DATE_RANGE_START'] = pd.to_datetime(df['SPEND_DATE_RANGE_START'])
    df['SPEND_DATE_RANGE_END'] = pd.to_datetime(df['SPEND_DATE_RANGE_END'])

    df_filtered_year = df_filtered[(df['SPEND_DATE_RANGE_START'] >= datetime_start) &
                                 (df['SPEND_DATE_RANGE_END'] <= datetime_end)]

    df_filtered = df_filtered_year.copy()

    # Convert string representations of lists in 'SPEND_BY_DAY' to actual lists
    df_filtered['SPEND_BY_DAY'] = df_filtered['SPEND_BY_DAY'].apply(ast.literal_eval)

    # Add a new column called 'DATE_OF_MONTH', as a list of integers from 1 to the end of the month (length of the 'SPEND_BY_DAY' list)
    df_filtered['DATE_OF_MONTH'] = df_filtered['SPEND_BY_DAY'].apply(lambda x: list(range(1, len(x) + 1)))

    df_spend_by_day = df_filtered.explode(['SPEND_BY_DAY', 'DATE_OF_MONTH'])

    df_spend_by_day['SPEND_BY_DAY'] = df_spend_by_day['SPEND_BY_DAY'].astype(float)
    df_spend_by_day['DATE_OF_MONTH'] = df_spend_by_day['DATE_OF_MONTH'].astype(int)

    df_spend_by_day['DATE'] = pd.to_datetime(df_spend_by_day['SPEND_DATE_RANGE_START']) + pd.to_timedelta(df_spend_by_day['DATE_OF_MONTH'] - 1, unit='D')

    df_grouped = df_spend_by_day.groupby('DATE')['SPEND_BY_DAY'].sum()

    return df_grouped

In [5]:
def filtermask_towns(df, towns):
    if len(towns) == 1:
        combined_mask = df['CITY'] == towns[0]
    else:
        combined_mask = np.zeros(len(df),dtype=bool)
        for i in towns:
            mask = df['CITY'] == i
            combined_mask += mask
    return combined_mask

In [6]:
towns = ['Gardiner', 'Yellowstone National Park', 'West Yellowstone', 'Cody', 'Cooke City', 'Silver Gate', 'Moran', 'Moose', 'Kelly', 'Teton Village', 'Wilson', 'Jackson']

region_mask = filtermask_towns(df_safegraph_poi, towns)
df_region = df_safegraph_poi[region_mask]

gardiner_mask = filtermask_towns(df_safegraph_poi, ['Gardiner'])
df_gardiner = df_safegraph_poi[gardiner_mask]
park_mask = filtermask_towns(df_safegraph_poi, ['Yellowstone National Park'])
df_park = df_safegraph_poi[park_mask]
weyel_mask = filtermask_towns(df_safegraph_poi, ['West Yellowstone'])
df_weyel = df_safegraph_poi[weyel_mask]
cody_mask = filtermask_towns(df_safegraph_poi, ['Cody'])
df_cody = df_safegraph_poi[cody_mask]

jackson = ['Moran', 'Moose', 'Kelly', 'Teton Village', 'Wilson', 'Jackson']
jackson_mask = filtermask_towns(df_safegraph_poi, jackson)
df_jackson = df_region[jackson_mask]

cooke_city = ['Cooke City', 'Silver Gate']
cooke_mask = filtermask_towns(df_safegraph_poi, cooke_city)
df_cooke = df_safegraph_poi[cooke_mask]

print('POIs in Gardiner: ', len(df_gardiner))
print('POIs in Yellowstone National Park: ', len(df_park))
print('POIs in West Yellowstone: ', len(df_weyel))
print('POIs in Cody: ', len(df_cody))
print('POIs in Jackson: ', len(df_jackson))
print('POIs in Cooke City-Silver Gate: ', len(df_cooke))


fig = figure()
fig = map_points(fig,df_region)
fig.update_layout(title="POIs in Greater Yellowstone Region",height=800,width=1200,mapbox_style="open-street-map")

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\401357287.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_jackson = df_region[jackson_mask]


POIs in Gardiner:  165
POIs in Yellowstone National Park:  109
POIs in West Yellowstone:  407
POIs in Cody:  1526
POIs in Jackson:  2550
POIs in Cooke City-Silver Gate:  54


<Figure size 640x480 with 0 Axes>

In [7]:
accomodation_naics = '721'
mask = [ str(ncode)[:3]==accomodation_naics for ncode in list(df_region['NAICS_CODE']) ]
df_hotels = df_region[mask]

food_naics = '722'
mask = [ str(ncode)[:3]==food_naics for ncode in list(df_region['NAICS_CODE']) ]
df_food = df_region[mask]

transit_naics = ['447', '485', '481']
combined_mask = np.zeros(len(df_region),dtype=bool)
for i in transit_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_region['NAICS_CODE']) ]
    combined_mask += mask
df_transit = df_region[combined_mask]

# Retail includes shopping and activites in the area including museums and amusement parks
retail_naics = ['452', '445', '451', '448', '446', '711', '712', '713']
combined_mask = np.zeros(len(df_region),dtype=bool)
for i in retail_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_region['NAICS_CODE']) ]
    combined_mask += mask
df_retail = df_region[combined_mask]

In [8]:
fig = figure()
fig = map_points(fig,df_hotels)
fig.update_layout(title="Accomodations in Greater Yellowstone Region",height=800,width=1200,mapbox_style="open-street-map")

<Figure size 640x480 with 0 Axes>

In [9]:
fig2 = map_points(fig,df_food)
fig2.update_layout(title="Restaurants in Greater Yellowstone Region",height=800,width=1200,mapbox_style="open-street-map")

In [10]:
fig3 = map_points(fig,df_transit)
fig3.update_layout(title="Transit in Greater Yellowstone Region",height=800,width=1200,mapbox_style="open-street-map")

In [11]:
fig4 = map_points(fig,df_retail)
fig4.update_layout(title="Retail in Greater Yellowstone Region",height=800,width=1200,mapbox_style="open-street-map")

In [12]:
# Regional Spending 
if 'PLACEKEY' in df_safegraph_spend.columns:
    df_region_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_region['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_region_hotels = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_hotels['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_region_food = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_food['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_region_retail = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_retail['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_region_transit = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_transit['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

print('Total POIs in region:', len(df_region))
print('Accomodation POIs in region: ', len(df_hotels))
print('Food POIs in region: ', len(df_food))
print('Retail POIs in region: ', len(df_retail))
print('Transit POIs in region: ', len(df_transit))

region_spending = get_spending(df_region_spending)
region_spending_hotels = get_spending(df_region_hotels)
region_spending_food = get_spending(df_region_food)
region_spending_retail = get_spending(df_region_retail)
region_spending_transit = get_spending(df_region_transit)

Total POIs in region: 4811
Accomodation POIs in region:  350
Food POIs in region:  511
Retail POIs in region:  693
Transit POIs in region:  129


C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [81]:
# Region Figures

import plotly.graph_objects as go
from plotly.subplots import make_subplots

region_fig = make_subplots(rows=5, cols=1, shared_xaxes=True, 
                         subplot_titles=("Total Daily Spending", 
                                         "Accomodation Spending",
                                         "Food Spending",
                                         "Retail Spending",
                                         "Transit Spending"))
line = px.line(region_spending)
region_total_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Total Spending')
region_fig.add_trace(region_total_chart, row=1, col=1)
line = px.line(region_spending_hotels)
region_hotel_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Accomodation Spending')
region_fig.add_trace(region_hotel_chart, row=2, col=1)
line = px.line(region_spending_food)
region_food_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Food Spending')
region_fig.add_trace(region_food_chart, row=3, col=1)
line = px.line(region_spending_retail)
region_retail_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Retail Spending')
region_fig.add_trace(region_retail_chart, row=4, col=1)
line = px.line(region_spending_transit)
reigon_transit_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Transit Spending')
region_fig.add_trace(reigon_transit_chart, row=5, col=1)
region_fig.update_layout(showlegend=False, title_text="2022 Daily Spending in Yellowstone National Park and Gateway Cities", height=800)
region_fig.update_yaxes(title_text="$")
region_fig.update_xaxes(title_text="Date", row=5)

In [13]:
# Park NAICS Breakdown

mask = [ str(ncode)[:3]==accomodation_naics for ncode in list(df_park['NAICS_CODE']) ]
df_park_hotels = df_park[mask]

mask = [ str(ncode)[:3]==food_naics for ncode in list(df_park['NAICS_CODE']) ]
df_park_food = df_park[mask]

combined_mask = np.zeros(len(df_park),dtype=bool)
for i in transit_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_park['NAICS_CODE']) ]
    combined_mask += mask
df_park_transit = df_park[combined_mask]

combined_mask = np.zeros(len(df_park),dtype=bool)
for i in retail_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_park['NAICS_CODE']) ]
    combined_mask += mask
df_park_retail = df_park[combined_mask]

print('Total POIs in park:', len(df_park))
print('Accomodation POIs in park: ', len(df_park_hotels))
print('Food POIs in park: ', len(df_park_food))
print('Retail POIs in park: ', len(df_park_retail))
print('Transit POIs in park: ', len(df_park_transit))


Total POIs in park: 109
Accomodation POIs in park:  19
Food POIs in park:  29
Retail POIs in park:  32
Transit POIs in park:  8


In [14]:
# Park spending
 
if 'PLACEKEY' in df_safegraph_spend.columns:
    df_park_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_park['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_park_hotels_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_park_hotels['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_park_food_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_park_food['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_park_retail_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_park_retail['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_park_transit_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_park_transit['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

print(df_park_transit_spending.head(5))

park_spending = get_spending(df_park_spending)
park_spending_hotels = get_spending(df_park_hotels_spending)
park_spending_food = get_spending(df_park_food_spending)
park_spending_retail = get_spending(df_park_retail_spending)
park_spending_transit = get_spending(df_park_transit_spending)

Empty DataFrame
Columns: [PLACEKEY, SAFEGRAPH_BRAND_IDS, BRANDS, SPEND_DATE_RANGE_START, SPEND_DATE_RANGE_END, RAW_TOTAL_SPEND, RAW_NUM_TRANSACTIONS, RAW_NUM_CUSTOMERS, MEDIAN_SPEND_PER_TRANSACTION, MEDIAN_SPEND_PER_CUSTOMER, SPEND_PER_TRANSACTION_PERCENTILES, SPEND_BY_DAY, SPEND_PER_TRANSACTION_BY_DAY, SPEND_BY_DAY_OF_WEEK, DAY_COUNTS, SPEND_PCT_CHANGE_VS_PREV_MONTH, SPEND_PCT_CHANGE_VS_PREV_YEAR, ONLINE_TRANSACTIONS, ONLINE_SPEND, TRANSACTION_INTERMEDIARY, SPEND_BY_TRANSACTION_INTERMEDIARY, BUCKETED_CUSTOMER_FREQUENCY, MEAN_SPEND_PER_CUSTOMER_BY_FREQUENCY, BUCKETED_CUSTOMER_INCOMES, MEAN_SPEND_PER_CUSTOMER_BY_INCOME, CUSTOMER_HOME_CITY, RELATED_CROSS_SHOPPING_PHYSICAL_BRANDS_PCT, RELATED_CROSS_SHOPPING_ONLINE_MERCHANTS_PCT, RELATED_CROSS_SHOPPING_SAME_CATEGORY_BRANDS_PCT, RELATED_CROSS_SHOPPING_LOCAL_BRANDS_PCT, RELATED_WIRELESS_CARRIER_PCT, RELATED_STREAMING_CABLE_PCT, RELATED_DELIVERY_SERVICE_PCT, RELATED_RIDESHARE_SERVICE_PCT, RELATED_BUYNOWPAYLATER_SERVICE_PCT, RELATED_PAYMENT_PL

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [80]:
# Park Spending Plots


park_fig = make_subplots(rows=4, cols=1, shared_xaxes=True, 
                         subplot_titles=("Total Daily Spending", 
                                         "Accomodation Spending",
                                         "Food Spending",
                                         "Retail Spending"))
line = px.line(park_spending)
park_total_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Total Spending')
park_fig.add_trace(park_total_chart, row=1, col=1)
line = px.line(park_spending_hotels)
park_hotel_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Accomodation Spending')
park_fig.add_trace(park_hotel_chart, row=2, col=1)
line = px.line(park_spending_food)
park_food_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Food Spending')
park_fig.add_trace(park_food_chart, row=3, col=1)

# No Yellowstone National Park Transit Data Avaliable

line = px.line(park_spending_retail)
park_retail_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Retail Spending')
park_fig.add_trace(park_retail_chart, row=4, col=1)
park_fig.update_layout(showlegend=False, title_text="2022 Daily Spending in Yellowstone National Park", height=800)
park_fig.update_yaxes(title_text="$")
park_fig.update_xaxes(title_text="Date", row=5)


In [16]:
# Jackson area NAICS Breakdown

mask = [ str(ncode)[:3]==accomodation_naics for ncode in list(df_jackson['NAICS_CODE']) ]
df_jackson_hotels = df_jackson[mask]

mask = [ str(ncode)[:3]==food_naics for ncode in list(df_jackson['NAICS_CODE']) ]
df_jackson_food = df_jackson[mask]

combined_mask = np.zeros(len(df_jackson),dtype=bool)
for i in transit_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_jackson['NAICS_CODE']) ]
    combined_mask += mask
df_jackson_transit = df_jackson[combined_mask]

combined_mask = np.zeros(len(df_jackson),dtype=bool)
for i in retail_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_jackson['NAICS_CODE']) ]
    combined_mask += mask
df_jackson_retail = df_jackson[combined_mask]

print('Total POIs in Jackson:', len(df_jackson))
print('Accomodation POIs in Jackson: ', len(df_jackson_hotels))
print('Food POIs in Jackson: ', len(df_jackson_food))
print('Retail POIs in Jackson: ', len(df_jackson_retail))
print('Transit POIs in Jackson: ', len(df_jackson_transit))


Total POIs in Jackson: 2550
Accomodation POIs in Jackson:  115
Food POIs in Jackson:  237
Retail POIs in Jackson:  397
Transit POIs in Jackson:  72


In [18]:
# Jackson Spending

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_jackson_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_jackson['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_jackson_hotels_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_jackson_hotels['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_jackson_food_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_jackson_food['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_jackson_retail_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_jackson_retail['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_jackson_transit_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_jackson_transit['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

jackson_spending = get_spending(df_jackson_spending)
jackson_spending_hotels = get_spending(df_jackson_hotels_spending)
jackson_spending_food = get_spending(df_jackson_food_spending)
jackson_spending_retail = get_spending(df_jackson_retail_spending)
jackson_spending_transit = get_spending(df_jackson_transit_spending)

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [79]:
# Jackson Figures

jackson_fig = make_subplots(rows=5, cols=1, shared_xaxes=True, 
                         subplot_titles=("Total Daily Spending", 
                                         "Accomodation Spending",
                                         "Food Spending",
                                         "Retail Spending",
                                         "Transit Spending"))
line = px.line(jackson_spending)
jackson_total_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Total Spending')
jackson_fig.add_trace(jackson_total_chart, row=1, col=1)
line = px.line(jackson_spending_hotels)
jackson_hotel_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Accomodation Spending')
jackson_fig.add_trace(jackson_hotel_chart, row=2, col=1)
line = px.line(jackson_spending_food)
jackson_food_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Food Spending')
jackson_fig.add_trace(jackson_food_chart, row=3, col=1)
line = px.line(jackson_spending_retail)
jackson_retail_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Retail Spending')
jackson_fig.add_trace(jackson_retail_chart, row=4, col=1)
line = px.line(jackson_spending_transit)
jackson_transit_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Transit Spending')
jackson_fig.add_trace(jackson_transit_chart, row=5, col=1)
jackson_fig.update_layout(showlegend=False, title_text="2022 Daily Spending in Jackson, WY", height=800)
jackson_fig.update_yaxes(title_text="$")
jackson_fig.update_xaxes(title_text="Date", row=5)

In [21]:
# Cody NAICS Breakdown

mask = [ str(ncode)[:3]==accomodation_naics for ncode in list(df_cody['NAICS_CODE']) ]
df_cody_hotels = df_cody[mask]

mask = [ str(ncode)[:3]==food_naics for ncode in list(df_cody['NAICS_CODE']) ]
df_cody_food = df_cody[mask]

combined_mask = np.zeros(len(df_cody),dtype=bool)
for i in transit_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_cody['NAICS_CODE']) ]
    combined_mask += mask
df_cody_transit = df_cody[combined_mask]

combined_mask = np.zeros(len(df_cody),dtype=bool)
for i in retail_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_cody['NAICS_CODE']) ]
    combined_mask += mask
df_cody_retail = df_cody[combined_mask]

print('Total POIs in Cody:', len(df_cody))
print('Accomodation POIs in Cody: ', len(df_cody_hotels))
print('Food POIs in Cody: ', len(df_cody_food))
print('Retail POIs in Cody: ', len(df_cody_retail))
print('Transit POIs in Cody: ', len(df_cody_transit))

Total POIs in Cody: 1526
Accomodation POIs in Cody:  71
Food POIs in Cody:  104
Retail POIs in Cody:  168
Transit POIs in Cody:  23


In [22]:
# Cody Spending

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_cody_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_cody['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_cody_hotels_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_cody_hotels['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_cody_food_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_cody_food['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_cody_retail_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_cody_retail['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_cody_transit_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_cody_transit['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

cody_spending = get_spending(df_cody_spending)
cody_spending_hotels = get_spending(df_cody_hotels_spending)
cody_spending_food = get_spending(df_cody_food_spending)
cody_spending_retail = get_spending(df_cody_retail_spending)
cody_spending_transit = get_spending(df_cody_transit_spending)

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [78]:
# Cody Figures

cody_fig = make_subplots(rows=5, cols=1, shared_xaxes=True, 
                         subplot_titles=("Total Daily Spending", 
                                         "Accomodation Spending",
                                         "Food Spending",
                                         "Retail Spending",
                                         "Transit Spending"))
line = px.line(cody_spending)
cody_total_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Total Spending')
cody_fig.add_trace(cody_total_chart, row=1, col=1)
line = px.line(cody_spending_hotels)
cody_hotel_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Accomodation Spending')
cody_fig.add_trace(cody_hotel_chart, row=2, col=1)
line = px.line(cody_spending_food)
cody_food_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Food Spending')
cody_fig.add_trace(cody_food_chart, row=3, col=1)
line = px.line(cody_spending_retail)
cody_retail_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Retail Spending')
cody_fig.add_trace(cody_retail_chart, row=4, col=1)
line = px.line(cody_spending_transit)
cody_transit_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Transit Spending')
cody_fig.add_trace(cody_transit_chart, row=5, col=1)
cody_fig.update_layout(showlegend=False, title_text="2022 Daily Spending in Cody, WY", height=800)
cody_fig.update_yaxes(title_text="$")
cody_fig.update_xaxes(title_text="Date", row=5)

In [25]:
# West Yellowstone NAICS Breakdown

mask = [ str(ncode)[:3]==accomodation_naics for ncode in list(df_weyel['NAICS_CODE']) ]
df_weyel_hotels = df_weyel[mask]

mask = [ str(ncode)[:3]==food_naics for ncode in list(df_weyel['NAICS_CODE']) ]
df_weyel_food = df_weyel[mask]

combined_mask = np.zeros(len(df_weyel),dtype=bool)
for i in transit_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_weyel['NAICS_CODE']) ]
    combined_mask += mask
df_weyel_transit = df_weyel[combined_mask]

combined_mask = np.zeros(len(df_weyel),dtype=bool)
for i in retail_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_weyel['NAICS_CODE']) ]
    combined_mask += mask
df_weyel_retail = df_weyel[combined_mask]

print('Total POIs in West Yellowstone:', len(df_weyel))
print('Accomodation POIs in West Yellowstone: ', len(df_weyel_hotels))
print('Food POIs in West Yellowstone: ', len(df_weyel_food))
print('Retail POIs in West Yellowstone: ', len(df_weyel_retail))
print('Transit POIs in West Yellowstone: ', len(df_weyel_transit))

Total POIs in West Yellowstone: 407
Accomodation POIs in West Yellowstone:  90
Food POIs in West Yellowstone:  83
Retail POIs in West Yellowstone:  65
Transit POIs in West Yellowstone:  17


In [26]:
# West Yellowstone Spending

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_weyel_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_weyel['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_weyel_hotels_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_weyel_hotels['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_weyel_food_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_weyel_food['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_weyel_retail_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_weyel_retail['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_weyel_transit_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_weyel_transit['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

weyel_spending = get_spending(df_weyel_spending)
weyel_spending_hotels = get_spending(df_weyel_hotels_spending)
weyel_spending_food = get_spending(df_weyel_food_spending)
weyel_spending_retail = get_spending(df_weyel_retail_spending)
weyel_spending_transit = get_spending(df_weyel_transit_spending)

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [77]:
# West Yellowstone Figures

weyel_fig = make_subplots(rows=5, cols=1, shared_xaxes=True, 
                         subplot_titles=("Total Daily Spending", 
                                         "Accomodation Spending",
                                         "Food Spending",
                                         "Retail Spending",
                                         "Transit Spending"))
line = px.line(weyel_spending)
weyel_total_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Total Spending')
weyel_fig.add_trace(weyel_total_chart, row=1, col=1)
line = px.line(weyel_spending_hotels)
weyel_hotel_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Accomodation Spending')
weyel_fig.add_trace(weyel_hotel_chart, row=2, col=1)
line = px.line(weyel_spending_food)
weyel_food_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Food Spending')
weyel_fig.add_trace(weyel_food_chart, row=3, col=1)
line = px.line(weyel_spending_retail)
weyel_retail_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Retail Spending')
weyel_fig.add_trace(weyel_retail_chart, row=4, col=1)
line = px.line(weyel_spending_transit)
weyel_transit_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Transit Spending')
weyel_fig.add_trace(weyel_transit_chart, row=5, col=1)
weyel_fig.update_layout(showlegend=False, title_text="2022 Daily Spending in West Yellowstone, MT", height=800)
weyel_fig.update_yaxes(title_text="$")
weyel_fig.update_xaxes(title_text="Date", row=5)

In [28]:
# Gardiner NAICS Breakdown

mask = [ str(ncode)[:3]==accomodation_naics for ncode in list(df_gardiner['NAICS_CODE']) ]
df_gardiner_hotels = df_gardiner[mask]

mask = [ str(ncode)[:3]==food_naics for ncode in list(df_gardiner['NAICS_CODE']) ]
df_gardiner_food = df_gardiner[mask]

combined_mask = np.zeros(len(df_gardiner),dtype=bool)
for i in transit_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_gardiner['NAICS_CODE']) ]
    combined_mask += mask
df_gardiner_transit = df_gardiner[combined_mask]

combined_mask = np.zeros(len(df_gardiner),dtype=bool)
for i in retail_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_gardiner['NAICS_CODE']) ]
    combined_mask += mask
df_gardiner_retail = df_gardiner[combined_mask]

print('Total POIs in Gardiner:', len(df_gardiner))
print('Accomodation POIs in Gardiner: ', len(df_gardiner_hotels))
print('Food POIs in Gardiner: ', len(df_gardiner_food))
print('Retail POIs in Gardiner: ', len(df_gardiner_retail))
print('Transit POIs in Gardiner: ', len(df_gardiner_transit))

Total POIs in Gardiner: 165
Accomodation POIs in Gardiner:  38
Food POIs in Gardiner:  47
Retail POIs in Gardiner:  24
Transit POIs in Gardiner:  6


In [29]:
# Gardiner Spending

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_gardiner_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_gardiner['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_gardiner_hotels_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_gardiner_hotels['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_gardiner_food_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_gardiner_food['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_gardiner_retail_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_gardiner_retail['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_gardiner_transit_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_gardiner_transit['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

gardiner_spending = get_spending(df_gardiner_spending)
gardiner_spending_hotels = get_spending(df_gardiner_hotels_spending)
gardiner_spending_food = get_spending(df_gardiner_food_spending)
gardiner_spending_retail = get_spending(df_gardiner_retail_spending)
gardiner_spending_transit = get_spending(df_gardiner_transit_spending)

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [76]:
# Gardiner Figures

gardiner_fig = make_subplots(rows=5, cols=1, shared_xaxes=True, 
                         subplot_titles=("Total Daily Spending", 
                                         "Accomodation Spending",
                                         "Food Spending",
                                         "Retail Spending",
                                         "Transit Spending"))
line = px.line(gardiner_spending)
gardiner_total_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Total Spending')
gardiner_fig.add_trace(gardiner_total_chart, row=1, col=1)
line = px.line(gardiner_spending_hotels)
gardiner_hotel_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Accomodation Spending')
gardiner_fig.add_trace(gardiner_hotel_chart, row=2, col=1)
line = px.line(gardiner_spending_food)
gardiner_food_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Food Spending')
gardiner_fig.add_trace(gardiner_food_chart, row=3, col=1)
line = px.line(cody_spending_retail)
gardiner_retail_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Retail Spending')
gardiner_fig.add_trace(gardiner_retail_chart, row=4, col=1)
line = px.line(gardiner_spending_transit)
gardiner_transit_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Transit Spending')
gardiner_fig.add_trace(gardiner_transit_chart, row=5, col=1)
gardiner_fig.update_layout(showlegend=False, title_text="2022 Daily Spending in Gardiner, MT", height=800)
gardiner_fig.update_yaxes(title_text="$")
gardiner_fig.update_xaxes(title_text="Date", row=5)

In [30]:
# Cooke City-Silver Gate NAICS Breakdown

mask = [ str(ncode)[:3]==accomodation_naics for ncode in list(df_cooke['NAICS_CODE']) ]
df_cooke_hotels = df_cooke[mask]

mask = [ str(ncode)[:3]==food_naics for ncode in list(df_cooke['NAICS_CODE']) ]
df_cooke_food = df_cooke[mask]

combined_mask = np.zeros(len(df_cooke),dtype=bool)
for i in transit_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_cooke['NAICS_CODE']) ]
    combined_mask += mask
df_cooke_transit = df_cooke[combined_mask]

combined_mask = np.zeros(len(df_cooke),dtype=bool)
for i in retail_naics:
    mask = [ str(ncode)[:3]==i for ncode in list(df_cooke['NAICS_CODE']) ]
    combined_mask += mask
df_cooke_retail = df_cooke[combined_mask]

print('Total POIs in Cooke City-Silver Gate:', len(df_cooke))
print('Accomodation POIs in Cooke City-Silver Gate: ', len(df_cooke_hotels))
print('Food POIs in Cooke City-Silver Gate: ', len(df_cooke_food))
print('Retail POIs in Cooke City-Silver Gate: ', len(df_cooke_retail))
print('Transit POIs in Cooke City-Silver Gate: ', len(df_cooke_transit))

Total POIs in Cooke City-Silver Gate: 54
Accomodation POIs in Cooke City-Silver Gate:  17
Food POIs in Cooke City-Silver Gate:  11
Retail POIs in Cooke City-Silver Gate:  7
Transit POIs in Cooke City-Silver Gate:  3


In [35]:
# Cooke City-Silver Gate Spending

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_cooke_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_cooke['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_cooke_hotels_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_cooke_hotels['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_cooke_food_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_cooke_food['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_cooke_retail_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_cooke_retail['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

if 'PLACEKEY' in df_safegraph_spend.columns:
    df_cooke_transit_spending = df_safegraph_spend[df_safegraph_spend['PLACEKEY'].isin(df_cooke_transit['PLACEKEY'])]
else:
    print("Error: DataFrame does not contain 'PLACEKEY' column.")

cooke_spending = get_spending(df_cooke_spending)
cooke_spending_hotels = get_spending(df_cooke_hotels_spending)
cooke_spending_food = get_spending(df_cooke_food_spending)
cooke_spending_retail = get_spending(df_cooke_retail_spending)
cooke_spending_transit = get_spending(df_cooke_transit_spending)

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\tjbla\AppData\Local\Temp\ipykernel_20348\3953764746.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [75]:
# Cooke City-Silver Gate Figures

cooke_fig = make_subplots(rows=4, cols=1, shared_xaxes=True, 
                         subplot_titles=("Total Daily Spending", 
                                         "Accomodation Spending",
                                         "Food Spending",
                                         "Retail Spending",
                                         "Transit Spending"))
line = px.line(cody_spending)
cooke_total_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Total Spending')
cooke_fig.add_trace(cooke_total_chart, row=1, col=1)
line = px.line(cooke_spending_hotels)
cooke_hotel_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Accomodation Spending')
cooke_fig.add_trace(cooke_hotel_chart, row=2, col=1)
line = px.line(cooke_spending_food)
cooke_food_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Food Spending')
cooke_fig.add_trace(cooke_food_chart, row=3, col=1)

# Retail spending data not avalibale for 2022

line = px.line(cooke_spending_transit)
cooke_transit_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Transit Spending')
cooke_fig.add_trace(cooke_transit_chart, row=4, col=1)
cooke_fig.update_layout(showlegend=False, title_text="2022 Daily Spending in Cooke City/Silver Gate, MT", height=800)
cooke_fig.update_yaxes(title_text="$")
cooke_fig.update_xaxes(title_text="Date", row=5)

In [87]:
region_fig = make_subplots(rows=7, cols=1, shared_xaxes=True, 
                         subplot_titles=("Total Daily Spending", 
                                         "Yellowstone National Park",
                                         "Jackson, WY",
                                         "Cody, WY",
                                         "West Yellowstone, MT",
                                         "Gardiner, MT",
                                         "Cooke City/Silver Gate, MT"))
line = px.line(region_spending)
region_total_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Total Spending')
region_fig.add_trace(region_total_chart, row=1, col=1)
line = px.line(park_spending)
region_hotel_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Accomodation Spending')
region_fig.add_trace(region_hotel_chart, row=2, col=1)
line = px.line(jackson_spending)
region_food_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Food Spending')
region_fig.add_trace(region_food_chart, row=3, col=1)
line = px.line(cody_spending)
region_retail_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Retail Spending')
region_fig.add_trace(region_retail_chart, row=4, col=1)
line = px.line(weyel_spending)
reigon_transit_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Transit Spending')
region_fig.add_trace(reigon_transit_chart, row=5, col=1)
line = px.line(gardiner_spending)
reigon_transit_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Transit Spending')
region_fig.add_trace(reigon_transit_chart, row=6, col=1)
line = px.line(cooke_spending)
reigon_transit_chart = go.Scatter(x=line.data[0]['x'], y=line.data[0]['y'], name='Transit Spending')
region_fig.add_trace(reigon_transit_chart, row=7, col=1)
region_fig.update_layout(showlegend=False, title_text="2022 Daily Spending in Yellowstone National Park and Gateway Cities", height=800)
region_fig.update_yaxes(title_text="$")
region_fig.update_xaxes(title_text="Date", row=7)

In [69]:

hotels_fig = make_subplots(rows=7, cols=1, shared_xaxes=True, 
                         subplot_titles=("Region", 
                                         "Yellowstone National Park",
                                         "Jackson, WY",
                                         "Cody, WY",
                                         "West Yellowstone, MT",
                                         "Gardiner, MT",
                                         "Cooke City/Silver Gate, MT"))


hotels_fig.add_trace(region_hotel_chart, row=1, col=1)
hotels_fig.add_trace(park_hotel_chart, row=2, col=1)
hotels_fig.add_trace(jackson_hotel_chart, row=3, col=1)
hotels_fig.add_trace(cody_hotel_chart, row=4, col=1)
hotels_fig.add_trace(weyel_hotel_chart, row=5, col=1)
hotels_fig.add_trace(gardiner_hotel_chart, row=6, col=1)
hotels_fig.add_trace(cooke_hotel_chart, row = 7, col = 1)

hotels_fig.update_layout(showlegend=False, title_text="2022 Daily Spending on Accomodations", height=800)
hotels_fig.update_yaxes(title_text="$")
hotels_fig.update_xaxes(title_text="Date", row=7)

In [70]:

food_fig = make_subplots(rows=7, cols=1, shared_xaxes=True, 
                         subplot_titles=("Region", 
                                         "Yellowstone National Park",
                                         "Jackson, WY",
                                         "Cody, WY",
                                         "West Yellowstone, MT",
                                         "Gardiner, MT",
                                         "Cooke City/Silver Gate, MT"))


food_fig.add_trace(region_food_chart, row=1, col=1)
food_fig.add_trace(park_food_chart, row=2, col=1)
food_fig.add_trace(jackson_food_chart, row=3, col=1)
food_fig.add_trace(cody_food_chart, row=4, col=1)
food_fig.add_trace(weyel_food_chart, row=5, col=1)
food_fig.add_trace(gardiner_food_chart, row=6, col=1)
food_fig.add_trace(cooke_food_chart, row = 7, col = 1)

hotels_fig.update_layout(showlegend=False, title_text="2022 Daily Spending on Food", height=800)
hotels_fig.update_yaxes(title_text="$")
hotels_fig.update_xaxes(title_text="Date", row=7)

In [73]:

retail_fig = make_subplots(rows=6, cols=1, shared_xaxes=True, 
                         subplot_titles=("Region", 
                                         "Yellowstone National Park",
                                         "Jackson, WY",
                                         "Cody, WY",
                                         "West Yellowstone, MT",
                                         "Gardiner, MT"))


retail_fig.add_trace(region_retail_chart, row=1, col=1)
retail_fig.add_trace(park_retail_chart, row=2, col=1)
retail_fig.add_trace(jackson_retail_chart, row=3, col=1)
retail_fig.add_trace(cody_retail_chart, row=4, col=1)
retail_fig.add_trace(weyel_retail_chart, row=5, col=1)
retail_fig.add_trace(gardiner_retail_chart, row=6, col=1)

retail_fig.update_layout(showlegend=False, title_text="2022 Daily Spending on Retail", height=800)
retail_fig.update_yaxes(title_text="$")
retail_fig.update_xaxes(title_text="Date", row=6)

In [85]:

transit_fig = make_subplots(rows=6, cols=1, shared_xaxes=True, 
                         subplot_titles=("Region", 
                                         "Jackson, WY",
                                         "Cody, WY",
                                         "West Yellowstone, MT",
                                         "Gardiner, MT",
                                         "Cooke City/Silver Gate, MT"))


transit_fig.add_trace(reigon_transit_chart, row=1, col=1)
transit_fig.add_trace(jackson_transit_chart, row=2, col=1)
transit_fig.add_trace(cody_transit_chart, row=3, col=1)
transit_fig.add_trace(weyel_transit_chart, row=4, col=1)
transit_fig.add_trace(gardiner_transit_chart, row=5, col=1)
transit_fig.add_trace(cooke_transit_chart, row = 6, col = 1)

transit_fig.update_layout(showlegend=False, title_text="2022 Daily Spending on Transit", height=800)
transit_fig.update_yaxes(title_text="$")
transit_fig.update_xaxes(title_text="Date", row=6)